# Creating inset figures

The goal of this exercise is to create an inset figure within an larger figure. It will require some manual handling of matplotlib axes handles that you access from yt objects. 

The solution notebook ends up with two figures, an astro figure with Enzo_64:

![](solutions/figures/Plotting_02_02_InsetFigures_Enzo_64.png)

and a figure for a seismic tomography model (which requires yt_xarray, cartopy, netcdf4, scipy)

![](solutions/figures/Plotting_02_01_InsetFigures_map.png)


In [1]:
# general imports you should already have
import matplotlib.pyplot as plt
import numpy as np
import yt

## A zoom-box inset with Enzo_64

**The Goal**: Create an plot that includes a zoomed-out view of a projection plot within an inset figure.

In [1]:
import yt 

ds = yt.load_sample("Enzo_64")

yt : [INFO     ] 2025-07-11 16:05:56,031 Sample dataset found in '/home/chavlin/hdd/data/yt_data/yt_sample_sets/Enzo_64/DD0043/data0043'
yt : [INFO     ] 2025-07-11 16:05:56,286 Parameters: current_time              = 645.81707236914
yt : [INFO     ] 2025-07-11 16:05:56,286 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2025-07-11 16:05:56,287 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-07-11 16:05:56,288 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-07-11 16:05:56,288 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2025-07-11 16:05:56,289 Parameters: current_redshift          = 0.0013930880640796
yt : [INFO     ] 2025-07-11 16:05:56,289 Parameters: omega_lambda              = 0.7
yt : [INFO     ] 2025-07-11 16:05:56,289 Parameters: omega_matter              = 0.3
yt : [INFO     ] 2025-07-11 16:05:56,290 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-07-11 16:05:56,290 Parameters:

first, let's create a projection plot across the whole domain:

In [ ]:
p_whole_domain = << write code to project a field, e.g., ('gas', 'density') >>

and say we want to check out the cluster of high density regions in the upper left in a bit more detail:

In [ ]:
c = ds.arr([100,30,135], 'Mpc')
wid = ds.quan(40, 'Mpc')

p_zoom =  << create another projection plot zoomed in on c with the above width. 
             use the same color scale as the previous plot >>

Now, the goal is to use `p_zoom` as our base image and add `p_whole_domain` within a new matplotlib axes that exists within the `p_zoom` figure. 

To do this, we'll need to access the underlying matplotlib objects stored in the yt plot containers.

First, yt plot containers have dict-like access to the yt plot objects. To check the fields in a plot:


In [5]:
p_zoom.fields

[('gas', 'density')]

and to access the underyling matplotlib figure and axes handles:

In [6]:
mpl_fig = p_zoom[('gas', 'density')].figure
mpl_ax = p_zoom[('gas', 'density')].axes
print(type(mpl_fig), type(mpl_ax))

<class 'matplotlib.figure.Figure'> <class 'matplotlib.axes._axes.Axes'>


having acces to the underlying matplotlib objects opens up the whole matplotlib API!

In our case, we want to use the `figure.add_axes` function to add a new set of child axes, and then we want to re-assign the `p_whole_domain` to (1) use those axes and (2) use the figure for `p_zoom`:

In [ ]:
# first, hide the colorbar and axes for `p_whole_domain`:

<< write code to hide the colorbar and axes, using methods off of p_whole_domain >>

# now, add our new axes
zoom_box_axes = << set the canvas position of the new axes (see figure.add_axes) >>
ax2 = << write code to use add_axes from the underlying matplotlib figure to add a 
         set of axes to the p_zoom[('gas', 'density')] plot >>

# over-ride p_whole_domain axes and figure for ('gas', 'density') to 
# point to the new axes and the figure for p_zoom ('gas', 'density')

p_whole_domain.plots[('gas', 'density')].axes = ax2
p_whole_domain.plots[('gas', 'density')].figure = p_zoom.plots[('gas', 'density')].figure

# add an outline in the inset to indicate the zoom
# using annotate_line to draw 4 lines to show the 
# bounding box of the zoom

<< write code to use p_whole_domain.annotate_line to draw a box indicating where 
   the p_zoom plot limts are >> 


# sometimes you have to call render after these operations that update underlying matplotlib objects...
p_whole_domain.render()  

# always reset before the final show/render!!
ax2.set_position(zoom_box_axes)
p_zoom.show()

## And now for some mapping!

### extra dependencies

For this, you'll want some extra dependencies

```
python -m pip install cartopy 
python -m pip install yt_xarray 
python -m pip install netcdf4
python -m pip install scipy
```

cartopy for map projections in yt figures and yt_xarray for streamlining the netcdf loading to get a yt dataset (yt_xarray will also install xarray for you). 

You'll also want some data!

A seismic tomography model is available at https://girder.hub.yt/api/v1/item/68718512ed776d031cebfea2/download , you can use wget to fetch it with

```
$ wget https://yt2025data.hub.yt/geo/wUS-SH-2010_percent.nc
```

This is a seismic tomography model of the western US showing shear wave velocity anamolies: first order mapping is to temperature of the upper mantle with negative (slower than reference) values indicate higher temperatures and positive values (faster than reference) indicate lower temperatures. Some extreme slow values also require the presence of melt (i.e., magma). 

Data citation:

* Schmandt, B. and E. Humphreys. 2010a. “Complex subduction and small-scale convection revealed by body-wave tomography of the western United States mantle.” Earth and Planetary Science Letters, 297, 435-445, https://doi.org/10.1016/j.epsl.2010.06.047.

Distributed by SAGE/Earthscope at https://doi.org/10.17611/DP/9991760


In [9]:
# imports for loading the seismic tomography model
# not needed if you are working with a different 
# dataset
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import yt_xarray 
import xarray as xr

First, load the dataset:

In [10]:
vs_file = "wUS-SH-2010_percent.nc" # or wherever your dataset ended up
ds_xr = yt_xarray.open_dataset(vs_file)  # an xarray dataset
ds_yt = ds_xr.yt.load_grid()  # a yt dataset

yt_xarray : [INFO ] 2025-07-02 21:47:38,557:  Inferred geometry type is geodetic. To override, use ds.yt.set_geometry
yt_xarray : [INFO ] 2025-07-02 21:47:38,557:  Attempting to detect if yt_xarray will require field interpolation:
yt_xarray : [INFO ] 2025-07-02 21:47:38,557:      stretched grid detected: yt_xarray will interpolate.
yt : [INFO     ] 2025-07-02 21:47:38,587 Parameters: current_time              = 0.0
yt : [INFO     ] 2025-07-02 21:47:38,587 Parameters: domain_dimensions         = [ 18  92 121]
yt : [INFO     ] 2025-07-02 21:47:38,587 Parameters: domain_left_edge          = [  60.     27.5  -125.75]
yt : [INFO     ] 2025-07-02 21:47:38,587 Parameters: domain_right_edge         = [885.   50.5 -95.5]
yt : [INFO     ] 2025-07-02 21:47:38,588 Parameters: cosmological_simulation   = 0


Now, let's check how to access underlying matplotlib handles. First, create a SlicePlot for example:

In [ ]:
c = ds_yt.domain_center.copy()
c[0]=150

p = << write code that:
   creates a sliceplot of the "dvs" field normal to "depth"
   with the above c value for the plot center >>

# set a nice projection for this dataset                        
p.set_mpl_projection(('Robinson', () , {'central_longitude':float(c[2].value), 'globe':None} ))


<< optionally write code to:
    * not take the log of dvs
    * adjust the colormap limits to be +/-8 (these are percent seismic shear wave speed anomalies)
    * choose a different colormap (just dont use seismic)
>>

p.show()

Now, the `fields` attributes will be a list of fields that are in the plot container (we have just one in this case):

In [ ]:
p.fields

Chec out the type of axes you have this time:

For geographic plots in yt, using cartopy functionality typically requires accessing these attributes. For example, to to add reference shapefile outlines from NaturalEarth:

In [ ]:

<< write code to access the underlying axes>>.add_feature(cfeature.NaturalEarthFeature(
        'cultural', 'admin_1_states_provinces_lines', '10m',
        edgecolor='gray', facecolor='none'))
<< write code to access the underlying axes>>.add_feature(cfeature.NaturalEarthFeature(
            'cultural', 'admin_0_countries', '10m',
            edgecolor='black', facecolor='none')) 

p.show()

and then adjust extents (to zoom in on the yellowstone hotspot track):

In [ ]:
<< write code to access the underlying axes>>.set_extent((-119, -104, 40, 48))
p.show()

Now we're ready to create our full figure

In [ ]:
c = ds_yt.domain_center.copy()
c[0] = 150.

zoom_extents = (-119, -104, 40, 48)

def get_slice():
    # get the base slice for both plots
    p = yt.SlicePlot(ds_yt, "depth", 'dvs',center=c)
    p.set_mpl_projection(('Robinson', () , {'central_longitude':float(c[2].value), 'globe':None} ))
    p.set_log('dvs',False)
    p.set_zlim('dvs',-8, 8)
    p.set_cmap('dvs','PuOr')
    return p


def add_features(carto_ax):
    # add NaturalEarthFeature objects to a cartopy axis
    carto_ax.add_feature(cfeature.NaturalEarthFeature(
        'cultural', 'admin_1_states_provinces_lines', '10m',
        edgecolor='gray', facecolor='none'))
    carto_ax.add_feature(cfeature.NaturalEarthFeature(
            'cultural', 'admin_0_countries', '10m',
            edgecolor='black', facecolor='none'))    
     
# initialize our slice plot to the zoomed-in view
p = get_slice()
p.set_colorbar_label('dvs',"$\Delta\mathrm{V_s}$ (%)")
p.render()

# set the cartopy axes extents via set_extent on the 
# underlying cartopy GeoAxes

<< write code to set_extent on the underlying cartopy GeoAxes using zoom_extents from above >>

# add in our features
add_features(<< pull out the underlying GeoAxes for the figure >>)


# add a small child axis with the same projection
# (use add_axes as for the astro example but 
# pass it the projection keyword)
locator_box = [0.05, 0.65, 0.3, 0.3]
proj = << write code to access the projection attribute of the underlying GeoAxes for the figure >>
ax2 = << to add_axes to the underlying figure: remember for GeoAxes, add_axes also requires a projection 
         keyword argument >>

# get a new slice, covering the whole domain and adjust the axes and 
# figure references the child axis and existing figure
p2 = get_slice()
<< update the p2 plot axes to the new axes, and point the figure to the initial figure p >>

# hide colorbar for inset figure
p2.hide_colorbar()

# makes ure p2 has re-rendered before adding annotations on top 
p2.render()

# add our features 
add_features(ax2)

# manually plot a box to indicate zoom extent

<< write code to call the .plot() method on the underlying geoaxes to plot an indicator 
   box showing the zoom extents. remember geoaxes require a transform argument that 
   represents the projection the underlying data is defined in: use transform=ccrs.PlateCarree() >>

# the child axes position gets reset somewhere, so set it again here
ax2.set_position(locator_box) 

p.show()

## Extra credit 

Make it to the end? Need more plotting? 

Try to recreate the slice plot and line plots from the `Plots_01_CYOPA_02_Callbacks` notebook, but put the density and temperature line plots on a single extra axes overlaid on the off axis slice. 